<h3>Load Libraries</h3>

In [ ]:
# Standard library
import json
import datetime

# 3rd party
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Geospatial
import cartopy.crs as ccrs
import geopandas as gpd
import shapely

# ARIMA
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

<h3>Load Filters</h3>

In [ ]:
filter_list = [[-12.216796875000002,59.40036514079251],[25.576171875,59.40036514079251],[25.576171875,36.10237644873644],[-12.216796875000002,36.10237644873644],[-12.216796875000002,59.40036514079251]] # europe
# filter_list = [[-10.898437500000002,34.59704151614417],[34.10156250000001,34.59704151614417],[34.10156250000001,-35.17380831799957],[-10.898437500000002,-35.17380831799957],[-10.898437500000002,34.59704151614417]] # africa
# filter_list = [[-125.06835937500001,45.767522962149904],[-116.982421875,45.767522962149904],[-116.982421875,31.728167146023935],[-125.06835937500001,31.728167146023935],[-125.06835937500001,45.767522962149904]] # california
filters_str = json.dumps(filter_list, separators=(",", ":"))


<h3>Load Data</h3>

In [ ]:
response = requests.get(f"http://127.0.0.1:9999/earthquake")
response_dict = response.json()

<h3>Convert to DataFrame</h3>


In [ ]:
"""Convert response to pandas - whole object"""
eq_list = []
for eq in response_dict['data']:
    timestamp = datetime.datetime.fromtimestamp(eq['time']//1000)
    dataframe_fields = {
        "id": eq['id'],
        "mag": eq['mag'],
        "time": datetime.datetime.fromtimestamp(eq['time']//1000),
        "latitude": eq['geometry']['coordinates'][1],
        "longitude": eq['geometry']['coordinates'][0],
        "height": eq['geometry']['coordinates'][2],
    }
    eq_list.append(dataframe_fields)

df = pd.DataFrame(eq_list)
df

<h3>Save DataFrame to Pickle</h3>

In [ ]:
# pickling
df.to_pickle('./earthquakes_with_coordinates.pkl')

<h3>Load DataFrame from Pickle</h3>


In [ ]:
# unpickling
df = pd.read_pickle('./earthquakes_with_coordinates.pkl')
df

<h3>Filter data from 1996 to 2022 (full years)</h3>

In [ ]:
df = df[df['time'].dt.year.isin(range(1996, 2023))]
df

<h3>Save DataFrame to Pickle (1996-2022)</h3>


In [ ]:
df.to_pickle('./earthquakes_with_coordinates_1996_2022.pkl')

<h3>Load DataFrame from Pickle (1996-2022)</h3>


In [ ]:
df = pd.read_pickle('./earthquakes_with_coordinates_1996_2022.pkl')
df


<h3>Filter DataFrame to Pickle >=4.0(1996-2022)</h3>


In [ ]:
df = df[df['mag'] >= 3.0]
df

<h3>Save DataFrame to Pickle >=4.0 (1996-2022)</h3>


In [ ]:
df.to_pickle('./earthquakes_with_coordinates_1996_2022_mag_3.pkl')


<h3>Load DataFrame from Pickle >=4.0 (1996-2022)</h3>


In [ ]:
df = pd.read_pickle('./earthquakes_with_coordinates_1996_2022_mag_3.pkl')
df


<h3>Plot Data</h3>

In [ ]:
plt.figure(figsize=(25,6))
filtered_df = df

plt.scatter(filtered_df['time'], filtered_df['mag'], s=[3])
len(filtered_df)

<h3>Get day from datetime</h3>


In [ ]:
filtered_df['date'] = filtered_df['time'].dt.date
df['year_month'] = df['time'].dt.to_period('M').astype(str)
filtered_df

<h3>Group by day and count</h3>


In [ ]:
# get count per day
day_count = filtered_df.groupby(filtered_df['year_month']).size().reset_index(name='count')

plt.plot(day_count['year_month'], day_count['count'])

<h3>Make grid for prediction</h3>


In [ ]:
xmin, ymin, xmax, ymax = -180, -90, 180, 90

n_cells=20
cell_size = (xmax-xmin)/n_cells

grid_cells = []
for x0 in np.arange(xmin, xmax+cell_size, cell_size ):
    for y0 in np.arange(ymin, ymax+cell_size, cell_size):
        # bounds
        x1 = x0-cell_size
        y1 = y0+cell_size
        grid_cells.append(shapely.geometry.box(x0, y0, x1, y1)  )
cell = gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs='epsg:4326')
cell

<h3>Plot all grids</h3>

In [ ]:
pd.set_option("display.max_rows", None)
cell.drop_duplicates()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5),
                       subplot_kw={'projection': ccrs.Orthographic(central_longitude=180, central_latitude=0)})

# Add the GeoSeries on top of the globe
ax.set_global()
ax.coastlines()  # Optional: add coastlines for better context
for index, series in cell.iloc[:219].iterrows():
   cell.iloc[[index]].geometry.to_crs(epsg=4326).plot(ax=ax, transform=ccrs.PlateCarree(), color='blue', edgecolor='black', alpha=0.5, aspect=1)

# Display the plot
plt.show()

<h3>Save grid to Pickle</h3>


In [ ]:
cell.to_pickle('./grid_20.pkl')

<h3>Load grid from Pickle</h3>


In [ ]:
grid = pd.read_pickle('./grid_20.pkl')
grid

<h3>Convert latitude and longitude to POINT</h3>


In [ ]:
gdf = gpd.GeoDataFrame(filtered_df, geometry=gpd.points_from_xy(filtered_df['longitude'], filtered_df['latitude']), crs="EPSG:4326")
gdf

<h3>Save earthquakes to Pickle</h3>


In [10]:
gdf.to_pickle('./earthquakes_with_coordinates_1996_2022_mag_3_20_year_month.pkl')

<h3>Load earthquakes to Pickle</h3>



In [ ]:

gdf = pd.read_pickle('./earthquakes_with_coordinates_1996_2022_mag_3_20_year_month.pkl')
gdf

<h3>Add Polygon index to earthquakes</h3>



In [ ]:
gdf_with_polygon = gpd.sjoin(gdf, grid, how="left", op="within")
gdf_with_polygon

<h3>Group by polygon index and count</h3>



In [ ]:
polygon_index_with_count_per_day = gdf_with_polygon.groupby(['index_right', 'year_month']).size().reset_index(name='count')
polygon_index_with_count_per_day

<h3>For each Grid cell make list of all days from 1996 to 2022 and count how many earthquakes occured each day</h3>


In [ ]:
# Create date range from 1996 to 2022 with monthly frequency
date_range = pd.date_range(start='1996-01', end='2022-12', freq='M').strftime('%Y-%m')

# Get unique polygon indices from grid
unique_polygons = grid.index

# Create all combinations of polygon indices and dates
all_combinations = pd.MultiIndex.from_product([unique_polygons, date_range], 
                                            names=['index_right', 'year_month'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

# Merge with actual counts, filling NaN with 0
complete_counts = pd.merge(all_combinations_df, 
                         polygon_index_with_count_per_day,
                         on=['index_right', 'year_month'],
                         how='left')
complete_counts['count'] = complete_counts['count'].fillna(0)

# Sort by index_right and date
complete_counts = complete_counts.sort_values(['index_right', 'year_month'])
complete_counts



<h3>Add grid polygons based on index of polygon</h3>


In [ ]:
# Merge grid polygons with complete counts
complete_counts_with_geometry = pd.merge(
    complete_counts,
    grid.reset_index(),
    left_on='index_right',
    right_index=True,
    how='left'
)

# Convert to GeoDataFrame
gdf_complete = gpd.GeoDataFrame(
    complete_counts_with_geometry, 
    geometry='geometry',
    crs=grid.crs
)

gdf_complete


<h3>Save complete geodataframe (polygons and counts per day)to Pickle</h3>


In [27]:
gdf_complete.to_pickle('./grid_polygons_1996_2022_mag_3_20_with_year_month_counts.pkl')


<h3>Load complete geodataframe (polygons and counts per day) from Pickle</h3>


In [3]:
gdf_complete = pd.read_pickle('./grid_polygons_1996_2022_mag_3_20_with_year_month_counts.pkl')


<h3>Select California polygon </h3>

In [ ]:
california_polygon = gdf_complete.loc[gdf_complete['index_right'] == 202]
california_polygon

<h3>Save California geodataframe (polygons and counts per day)to Pickle</h3>



In [30]:
california_polygon.to_pickle('./california_polygon_1996_2022_mag_3_20_with_year_month_counts.pkl')


<h3>Load California geodataframe (polygons and counts per day) from Pickle</h3>

In [ ]:
california_polygon = pd.read_pickle('./california_polygon_1996_2022_mag_3_20_with_year_month_counts.pkl')
california_polygon

In [ ]:
avg_earthquakes = gdf_complete.groupby('index_right')['count'].mean().sort_values(ascending=False)
avg_earthquakes

# Get polygon with highest average
california_polygon = gdf_complete.loc[gdf_complete['index_right'] == avg_earthquakes.index[0]]
pd.options.display.max_rows = 10
california_polygon

<h3>Plot California polygon on map</h3>


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5),
                       subplot_kw={'projection': ccrs.Orthographic(central_longitude=120, central_latitude=0)})
                     #   subplot_kw={'projection': ccrs.Orthographic(central_longitude=-72, central_latitude=-36)})

# Add the GeoSeries on top of the globe
ax.set_global()
ax.coastlines()  # Optional: add coastlines for better context
california_polygon.head(1).plot(ax=ax, transform=ccrs.PlateCarree(), color='blue', edgecolor='black', alpha=0.5)

# Display the plot
plt.show()

<h3>Plot California earthquakes per day</h3>


In [ ]:
# Extract years from year_month and get unique values
# Filter for just 1996
df_1996 = california_polygon[pd.to_datetime(california_polygon['year_month']).dt.year == 1996]

# Get monthly counts for 1996
monthly_counts = df_1996.groupby('year_month')['count'].mean()

# Get month numbers for x-axis
months = pd.to_datetime(monthly_counts.index).month

plt.plot(months, monthly_counts)
plt.xlabel('Miesiąc')
plt.ylabel('Liczba trzęsień ziemi')
plt.title('Liczba trzęsień ziemi w kolejnych miesiącach w 1996 roku')

<h3>Prepare data for ARIMA prediction</h3>





In [7]:
df_for_arima = california_polygon[['year_month', 'count']]
df_for_arima = df_for_arima.reset_index(drop=True)

# Calculate split point at 80% of data
split_point = int(len(df_for_arima) * 0.8)

# Split into train (80%) and test (20%) sets
train = df_for_arima[:split_point]
test = df_for_arima[split_point:]


In [ ]:
train

<h3>Train model with AutoARIMA</h3>


In [ ]:
# model = pm.auto_arima(train['count'],
model = pm.auto_arima(train['count'],
                      # m=12,               # frequency of series
                      seasonal=False,  # TRUE if seasonal series
                      d=None,             # let model determine 'd'
                      test='adf',         # use adftest to find optimal 'd'
                      start_p=0, start_q=0, # minimum p and q
                      max_p=12, max_q=12, # maximum p and q 
                      D=None,             # let model determine 'D'
                      trace=True,
                      error_action='ignore',
                      suppress_warnings=True,
                      stepwise=True)

<h3>Model summary</h3>

In [ ]:
print(model.summary())

<h3>Plot predictions</h3>

In [9]:
test_count = int(len(df_for_arima) * 0.2) + 1
predicted, confint = model.predict(n_periods=test_count, return_conf_int=True)

In [ ]:
predicted

In [ ]:
test_plot = test
predicted_plot = predicted
print(test_plot)
days = test_plot['year_month']
plt.plot(df_for_arima['year_month'], df_for_arima['count'], label='Wartości rzeczywiste')
plt.plot(days, predicted_plot.iloc[:], label='Wartości przewidziane')
plt.legend()

In [ ]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test_plot['count'], predicted_plot))
print(f'Root Mean Square Error (RMSE): {rmse:.2f}')

# Calculate MAE 
mae = mean_absolute_error(test_plot['count'], predicted_plot)
print(f'Mean Absolute Error (MAE): {mae:.2f}')